In [1]:
import pandas as pd
import numpy as np
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 먼저 역 위,경도데이터
station = pd.read_csv('data/station_coordinate.csv')
station.to_excel('data/station_total.xlsx')

In [3]:
# 결측치는 직접 찾아 넣었음
station = pd.read_excel('data/station_total.xlsx')
station

,Unnamed: 0,line,name,code,lat,lng
0,0,01호선,녹양,1908.0,37.759380,127.042292
1,1,01호선,남영,1002.0,37.541021,126.971300
2,2,01호선,용산,1003.0,37.529849,126.964561
3,3,01호선,노량진,1004.0,37.514219,126.942454
4,4,01호선,대방,1005.0,37.513342,126.926382
...,...,...,...,...,...,...
725,725,인천선,지식정보단지,3135.0,37.378384,126.645168
726,726,인천선,인천대입구,3136.0,37.386007,126.639484
727,727,인천선,센트럴파크,3137.0,37.393054,126.634729
728,728,인천선,국제업무지구,3138.0,37.399907,126.630347


In [4]:
# 아파트 위경도데이터(ID있음)
where = pd.read_excel('data/03_결측치처리후_주소지_4천개.xlsx')
where.drop(['Unnamed: 0'], axis=1, inplace=True)

# 이하 데이터 전처리
base = pd.read_excel('data/ID있는버전.xlsx')
base['면적당매매가'] = base['거래금액(만원)'] / base['전용면적(㎡)']
base['저층여부'] = np.where(base['층']<5, '저층', '고층')
base['계약년월'] = pd.to_datetime(base['계약년월'], format='%Y%m')
base['분기'], base['년도'] = (base['계약년월'].dt.quarter, base['계약년월'].dt.year)
base['분기'] = base[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
del base['년도']
del base['계약일']
base.drop(base[base['전용면적(㎡)']<=10].index, inplace=True)

# 거래금액, 전용면적, 건축년도 모두 사용해 보기로 함
base['계약년월'] = pd.to_datetime(base['계약년월'], format='%Y%m')
base['분기'], base['년도'] = (base['계약년월'].dt.quarter, base['계약년월'].dt.year)
base['분기'] = base[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
del base['년도']
table1 = pd.pivot_table(base, values=['거래금액(만원)'], index=['분기'], columns=['ID'], aggfunc=np.mean)
table2 = pd.pivot_table(base, values=['전용면적(㎡)'], index=['분기'], columns=['ID'], aggfunc=np.mean)
table3 = pd.pivot_table(base, values=['건축년도'], index=['분기'], columns=['ID'], aggfunc=np.mean)
check_null1 = table1.isna().sum() / 18
check_null2 = table2.isna().sum() / 18
check_null3 = table3.isna().sum() / 18
check_null1[check_null1 >= 0.5]
check_null2[check_null2 >= 0.5]
check_null3[check_null3 >= 0.5]
remove_cols1 = check_null1[check_null1 >= 0.5].keys()
remove_cols2 = check_null2[check_null2 >= 0.5].keys()
remove_cols3 = check_null3[check_null3 >= 0.5].keys()
table1 = table1.drop(remove_cols1, axis=1)
table2 = table2.drop(remove_cols2, axis=1)
table3 = table3.drop(remove_cols3, axis=1)
table1.fillna(method='ffill', inplace=True)
table1.fillna(method='bfill', inplace=True)
table2.fillna(method='ffill', inplace=True)
table2.fillna(method='bfill', inplace=True)
table3.fillna(method='ffill', inplace=True)
table3.fillna(method='bfill', inplace=True)
data1 = table1.transpose()
data2 = table2.transpose()
data3 = table3.transpose()
money = data1.droplevel(axis=0, level=0)
width = data2.droplevel(axis=0, level=0)
year = data3.droplevel(axis=0, level=0)
data = pd.concat([money,width,year], axis=1)
data = data.round(1)

In [5]:
# 데이터 결합
dic = {}
number = 0

for i1 in range(1,19):
    dic[f'data{i1}']=data.iloc[:,[number, number+18, number+36]]
    number += 1

In [6]:
# 확인
dic['data1']

분기,2018/1,2018/1,2018/1
ID,,,
서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,80.0,1987.0
서울특별시 강남구 개포동 개포주공1단지 40,145736.8,40.0,1982.0
서울특별시 강남구 개포동 개포주공5단지 60,136666.7,60.0,1983.0
서울특별시 강남구 개포동 개포주공5단지 80,177416.7,80.0,1983.0
서울특별시 강남구 개포동 개포주공6단지 50,125900.0,50.0,1983.0
...,...,...,...
서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,60.0,1998.0
서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,80.0,2005.0
서울특별시 중랑구 중화동 한신아파트 50,31756.2,50.0,1997.0


In [7]:
# 출발지점과 도착지점 저장
stay_df = pd.DataFrame({'위도':station['lat'].values,'경도':station['lng'].values})
address_df = pd.DataFrame({'위도':where['위도'].values,'경도':where['경도'].values})
stay_df, address_df

(            위도          경도
 0    37.759380  127.042292
 1    37.541021  126.971300
 2    37.529849  126.964561
 3    37.514219  126.942454
 4    37.513342  126.926382
 ..         ...         ...
 725  37.378384  126.645168
 726  37.386007  126.639484
 727  37.393054  126.634729
 728  37.399907  126.630347
 729  37.467048  126.707938
 
 [730 rows x 2 columns],
              위도          경도
 0     37.483372  127.053437
 1     37.483381  127.057068
 2     37.487868  127.068988
 3     37.487868  127.068988
 4     37.488481  127.072502
 ...         ...         ...
 4089  37.605430  127.082936
 4090  37.599447  127.078938
 4091  37.595961  127.080314
 4092  37.595961  127.080314
 4093  37.595961  127.080314
 
 [4094 rows x 2 columns])

In [8]:
# 거리계산
g = []

for i in range(len(stay_df)):
    st_name = station['name'][i]
    st_line = station['line'][i]
    start = (float(stay_df['위도'][i]), float(stay_df['경도'][i]))
    for j in range(len(address_df)):
        apt_name = where['ID'][j]
        goal = (float(address_df['위도'][j]), float(address_df['경도'][j]))
        geoly = haversine(start, goal)
        r_start = np.round(start, 2)
        r_goal = np.round(goal, 2)
        r_geoly = np.round(geoly, 2)
        if r_geoly <=0.5:
            g.append([st_name, st_line, r_start, apt_name, r_goal, r_geoly])
        else:
            pass

In [9]:
# 데이터프레임화
g_df = pd.DataFrame(g, columns=['역 이름', '역 호선', '역 좌표', '아파트 이름', '아파트 좌표', '역까지의 거리'])
g_df.head()

,역 이름,역 호선,역 좌표,아파트 이름,아파트 좌표,역까지의 거리
0,남영,01호선,"[37.54, 126.97]",서울특별시 용산구 문배동 용산KCC웰츠타워 80,"[37.54, 126.97]",0.49
1,남영,01호선,"[37.54, 126.97]",서울특별시 용산구 문배동 이안용산1차 80,"[37.54, 126.97]",0.49
2,남영,01호선,"[37.54, 126.97]",서울특별시 용산구 원효로1가 용산더프라임 80,"[37.54, 126.97]",0.22
3,용산,01호선,"[37.53, 126.96]",서울특별시 용산구 한강로2가 벽산메가트리움 80,"[37.53, 126.97]",0.45
4,용산,01호선,"[37.53, 126.96]",서울특별시 용산구 한강로2가 한강로대우아이빌 30,"[37.53, 126.97]",0.45


In [10]:
# 검색 실증
def search(x):
    x = g_df[g_df['아파트 이름']==x]
    return x
len(search('미국'))

0

In [11]:
# 실험1
base['ID'].iloc[0]

'서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80'

In [12]:
# 실험2
dic['data1'].iloc[1]

분기
2018/1    145736.8
2018/1        40.0
2018/1      1982.0
Name: 서울특별시 강남구 개포동 개포주공1단지 40, dtype: float64

In [13]:
# 인접역 산출
p1 = []
n = 0
for i in range(len(base['ID'])):
    a = base['ID'].iloc[n]
    b = len(search(base['ID'].iloc[n]))
    p1.append([a,b])
    n+=1
# test = pd.concat([dic['data1'], p1], axis=1)
p1_df = pd.DataFrame(p1, columns=['ID','인접역 수'])

In [14]:
# 인접역 산출 결과
p1_df.head()

,ID,인접역 수
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,0
1,서울특별시 동작구 신대방동 롯데관악타워 150,0
2,서울특별시 동작구 신대방동 경남교수 80,0
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,1
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,1


In [15]:
# 길이
len(p1_df)

281197

In [16]:
# 각 분기별 별도의 데이터프레임화, 컬럼명 붙이기, 인접역 붙이기, 중복값 제거
# 데이터프레임화를 안하면 이상한 오류가 나기도 함
# for문 활용시 인접역 붙이기에서 오류가 나는데 해결을 못함
# 중복값을 제거하지 않으면 파악하지 못한 이유로 데이터가 약 20만행 나옴
tmp1 = pd.DataFrame(dic['data1'])
tmp1.reset_index(inplace=True)
tmp1.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main1 = pd.merge(tmp1, p1_df)
main1.drop_duplicates(inplace=True)

tmp2 = pd.DataFrame(dic['data2'])
tmp2.reset_index(inplace=True)
tmp2.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main2 = pd.merge(tmp2, p1_df)
main2.drop_duplicates(inplace=True)

tmp3 = pd.DataFrame(dic['data3'])
tmp3.reset_index(inplace=True)
tmp3.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main3 = pd.merge(tmp3, p1_df)
main3.drop_duplicates(inplace=True)

tmp4 = pd.DataFrame(dic['data4'])
tmp4.reset_index(inplace=True)
tmp4.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main4 = pd.merge(tmp4, p1_df)
main4.drop_duplicates(inplace=True)

tmp5 = pd.DataFrame(dic['data5'])
tmp5.reset_index(inplace=True)
tmp5.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main5 = pd.merge(tmp5, p1_df)
main5.drop_duplicates(inplace=True)

tmp6 = pd.DataFrame(dic['data6'])
tmp6.reset_index(inplace=True)
tmp6.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main6 = pd.merge(tmp6, p1_df)
main6.drop_duplicates(inplace=True)

tmp7 = pd.DataFrame(dic['data7'])
tmp7.reset_index(inplace=True)
tmp7.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main7 = pd.merge(tmp7, p1_df)
main7.drop_duplicates(inplace=True)

tmp8 = pd.DataFrame(dic['data8'])
tmp8.reset_index(inplace=True)
tmp8.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main8 = pd.merge(tmp8, p1_df)
main8.drop_duplicates(inplace=True)

tmp9 = pd.DataFrame(dic['data9'])
tmp9.reset_index(inplace=True)
tmp9.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main9 = pd.merge(tmp9, p1_df)
main9.drop_duplicates(inplace=True)

tmp10 = pd.DataFrame(dic['data10'])
tmp10.reset_index(inplace=True)
tmp10.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main10 = pd.merge(tmp10, p1_df)
main10.drop_duplicates(inplace=True)

tmp11 = pd.DataFrame(dic['data11'])
tmp11.reset_index(inplace=True)
tmp11.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main11 = pd.merge(tmp11, p1_df)
main11.drop_duplicates(inplace=True)

tmp12 = pd.DataFrame(dic['data12'])
tmp12.reset_index(inplace=True)
tmp12.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main12 = pd.merge(tmp12, p1_df)
main12.drop_duplicates(inplace=True)

tmp13 = pd.DataFrame(dic['data13'])
tmp13.reset_index(inplace=True)
tmp13.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main13 = pd.merge(tmp13, p1_df)
main13.drop_duplicates(inplace=True)

tmp14 = pd.DataFrame(dic['data14'])
tmp14.reset_index(inplace=True)
tmp14.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main14 = pd.merge(tmp14, p1_df)
main14.drop_duplicates(inplace=True)

tmp15 = pd.DataFrame(dic['data15'])
tmp15.reset_index(inplace=True)
tmp15.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main15 = pd.merge(tmp15, p1_df)
main15.drop_duplicates(inplace=True)

tmp16 = pd.DataFrame(dic['data16'])
tmp16.reset_index(inplace=True)
tmp16.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main16 = pd.merge(tmp16, p1_df)
main16.drop_duplicates(inplace=True)

tmp17 = pd.DataFrame(dic['data17'])
tmp17.reset_index(inplace=True)
tmp17.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main17 = pd.merge(tmp17, p1_df)
main17.drop_duplicates(inplace=True)

tmp18 = pd.DataFrame(dic['data18'])
tmp18.reset_index(inplace=True)
tmp18.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
main18 = pd.merge(tmp18, p1_df)
main18.drop_duplicates(inplace=True)

In [17]:
# 확인
main18

,ID,거래금액(만원),전용면적,건축년도,인접역 수
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,218666.7,80.0,1987.0,0
24,서울특별시 강남구 개포동 개포주공1단지 40,180000.0,40.0,1982.0,1
146,서울특별시 강남구 개포동 개포주공5단지 60,202800.0,60.0,1983.0,1
171,서울특별시 강남구 개포동 개포주공5단지 80,282250.0,80.0,1983.0,1
199,서울특별시 강남구 개포동 개포주공6단지 50,189000.0,50.0,1983.0,1
...,...,...,...,...,...
214320,서울특별시 중랑구 중화동 중화동건영아파트 60,47500.0,60.0,1998.0,1
214339,서울특별시 중랑구 중화동 태능에셈빌 80,71000.0,80.0,2005.0,1
214360,서울특별시 중랑구 중화동 한신아파트 50,70000.0,50.0,1997.0,5
214443,서울특별시 중랑구 중화동 한신아파트 60,73000.0,60.0,1997.0,5


In [18]:
# ID제거, ID가 문자열이라서 스케일링에 문제가 생김
main1.drop(columns='ID', inplace=True)
main2.drop(columns='ID', inplace=True)
main3.drop(columns='ID', inplace=True)
main4.drop(columns='ID', inplace=True)
main5.drop(columns='ID', inplace=True)
main6.drop(columns='ID', inplace=True)
main7.drop(columns='ID', inplace=True)
main8.drop(columns='ID', inplace=True)
main9.drop(columns='ID', inplace=True)
main10.drop(columns='ID', inplace=True)
main11.drop(columns='ID', inplace=True)
main12.drop(columns='ID', inplace=True)
main13.drop(columns='ID', inplace=True)
main14.drop(columns='ID', inplace=True)
main15.drop(columns='ID', inplace=True)
main16.drop(columns='ID', inplace=True)
main17.drop(columns='ID', inplace=True)
main18.drop(columns='ID', inplace=True)

In [19]:
# 스케일링과 클러스터링
# 여전히 군집고정은 해결 못함
mm = MinMaxScaler()
mm_scaled1 = mm.fit_transform(main1)
mm_scaled2 = mm.fit_transform(main2)
mm_scaled3 = mm.fit_transform(main3)
mm_scaled4 = mm.fit_transform(main4)
mm_scaled5 = mm.fit_transform(main5)
mm_scaled6 = mm.fit_transform(main6)
mm_scaled7 = mm.fit_transform(main7)
mm_scaled8 = mm.fit_transform(main8)
mm_scaled9 = mm.fit_transform(main9)
mm_scaled10 = mm.fit_transform(main10)
mm_scaled11 = mm.fit_transform(main11)
mm_scaled12 = mm.fit_transform(main12)
mm_scaled13 = mm.fit_transform(main13)
mm_scaled14 = mm.fit_transform(main14)
mm_scaled15 = mm.fit_transform(main15)
mm_scaled16 = mm.fit_transform(main16)
mm_scaled17 = mm.fit_transform(main17)
mm_scaled18 = mm.fit_transform(main18)

kmeans = KMeans(n_clusters=5, init='k-means++', random_state=np.random.RandomState(seed=1))

clusters1 = kmeans.fit(mm_scaled1)
main1['cluster'] = clusters1.labels_
clusters2 = kmeans.fit(mm_scaled2)
main2['cluster'] = clusters2.labels_
clusters3 = kmeans.fit(mm_scaled3)
main3['cluster'] = clusters3.labels_
clusters4 = kmeans.fit(mm_scaled4)
main4['cluster'] = clusters4.labels_
clusters5 = kmeans.fit(mm_scaled5)
main5['cluster'] = clusters5.labels_
clusters6 = kmeans.fit(mm_scaled6)
main6['cluster'] = clusters6.labels_
clusters7 = kmeans.fit(mm_scaled7)
main7['cluster'] = clusters7.labels_
clusters8 = kmeans.fit(mm_scaled8)
main8['cluster'] = clusters8.labels_
clusters9 = kmeans.fit(mm_scaled9)
main9['cluster'] = clusters9.labels_
clusters10 = kmeans.fit(mm_scaled10)
main10['cluster'] = clusters10.labels_
clusters11 = kmeans.fit(mm_scaled11)
main11['cluster'] = clusters11.labels_
clusters12 = kmeans.fit(mm_scaled12)
main12['cluster'] = clusters12.labels_
clusters13 = kmeans.fit(mm_scaled13)
main13['cluster'] = clusters13.labels_
clusters14 = kmeans.fit(mm_scaled14)
main14['cluster'] = clusters14.labels_
clusters15 = kmeans.fit(mm_scaled15)
main15['cluster'] = clusters15.labels_
clusters16 = kmeans.fit(mm_scaled16)
main16['cluster'] = clusters16.labels_
clusters17 = kmeans.fit(mm_scaled17)
main17['cluster'] = clusters17.labels_
clusters18 = kmeans.fit(mm_scaled18)
main18['cluster'] = clusters18.labels_

In [20]:
# 확인
main18

,거래금액(만원),전용면적,건축년도,인접역 수,cluster
0,218666.7,80.0,1987.0,0,1
24,180000.0,40.0,1982.0,1,1
146,202800.0,60.0,1983.0,1,1
171,282250.0,80.0,1983.0,1,1
199,189000.0,50.0,1983.0,1,1
...,...,...,...,...,...
214320,47500.0,60.0,1998.0,1,3
214339,71000.0,80.0,2005.0,1,2
214360,70000.0,50.0,1997.0,5,4
214443,73000.0,60.0,1997.0,5,4


In [24]:
# ID 다시 붙여주기
main1.set_index(tmp1['ID'], inplace=True)

In [25]:
main2.set_index(tmp1['ID'], inplace=True)
main3.set_index(tmp1['ID'], inplace=True)
main4.set_index(tmp1['ID'], inplace=True)
main5.set_index(tmp1['ID'], inplace=True)
main6.set_index(tmp1['ID'], inplace=True)
main7.set_index(tmp1['ID'], inplace=True)
main8.set_index(tmp1['ID'], inplace=True)
main9.set_index(tmp1['ID'], inplace=True)
main10.set_index(tmp1['ID'], inplace=True)
main11.set_index(tmp1['ID'], inplace=True)
main12.set_index(tmp1['ID'], inplace=True)
main13.set_index(tmp1['ID'], inplace=True)
main14.set_index(tmp1['ID'], inplace=True)
main15.set_index(tmp1['ID'], inplace=True)
main16.set_index(tmp1['ID'], inplace=True)
main17.set_index(tmp1['ID'], inplace=True)
main18.set_index(tmp1['ID'], inplace=True)

In [26]:
# 확인
main18

,거래금액(만원),전용면적,건축년도,인접역 수,cluster
ID,,,,,
서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,218666.7,80.0,1987.0,0,1
서울특별시 강남구 개포동 개포주공1단지 40,180000.0,40.0,1982.0,1,1
서울특별시 강남구 개포동 개포주공5단지 60,202800.0,60.0,1983.0,1,1
서울특별시 강남구 개포동 개포주공5단지 80,282250.0,80.0,1983.0,1,1
서울특별시 강남구 개포동 개포주공6단지 50,189000.0,50.0,1983.0,1,1
...,...,...,...,...,...
서울특별시 중랑구 중화동 중화동건영아파트 60,47500.0,60.0,1998.0,1,3
서울특별시 중랑구 중화동 태능에셈빌 80,71000.0,80.0,2005.0,1,2
서울특별시 중랑구 중화동 한신아파트 50,70000.0,50.0,1997.0,5,4


In [27]:
# 하나의 데이터프레임으로 저장
address = pd.merge(main1, where, left_on='ID', right_on='ID', how='inner')
address

,ID,거래금액(만원),전용면적,건축년도,인접역 수,cluster,위도,경도
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,80.0,1987.0,0,4,37.483372,127.053437
1,서울특별시 강남구 개포동 개포주공1단지 40,145736.8,40.0,1982.0,1,4,37.483381,127.057068
2,서울특별시 강남구 개포동 개포주공5단지 60,136666.7,60.0,1983.0,1,4,37.487868,127.068988
3,서울특별시 강남구 개포동 개포주공5단지 80,177416.7,80.0,1983.0,1,4,37.487868,127.068988
4,서울특별시 강남구 개포동 개포주공6단지 50,125900.0,50.0,1983.0,1,4,37.488481,127.072502
...,...,...,...,...,...,...,...,...
4089,서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,60.0,1998.0,1,2,37.605430,127.082936
4090,서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,80.0,2005.0,1,3,37.599447,127.078938
4091,서울특별시 중랑구 중화동 한신아파트 50,31756.2,50.0,1997.0,5,1,37.595961,127.080314
4092,서울특별시 중랑구 중화동 한신아파트 60,36534.6,60.0,1997.1,5,1,37.595961,127.080314


In [30]:
address = pd.merge(main2, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main3, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main4, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main5, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main6, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main7, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main8, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main9, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main10, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main11, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main12, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main13, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main14, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main15, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main16, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main17, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(main18, address, left_on='ID', right_on='ID', how='inner')

In [31]:
# 데이터프레임 엑셀로 저장
address.to_excel('data/Mega_address.xlsx')

이후 엑셀에서 전용면적, 건축년도, 인접역 등 시간에 따라 변화하지 않는 피쳐들이 중복되는 부분을 지우고 컬럼 이름을 조정함  
군집화는 해당 피쳐가 다 들어있는 상태로 시행됨  
최적 군집 갯수 테스트는 따로 안해봄, 어차피 피쳐를 더 넣으면 또 해야함  